In [1]:
import os
import time
import timeit
import redis
from tqdm import tqdm
from faker import Faker

In [2]:
master_port = 6379
branch_port = 6378

In [3]:
prefix = "/home/ubuntu/code/"
workdir = "RediSearchClean"  # Master build

# Remove dump file if it exists
if os.path.exists(f"{os.getcwd()}/dump.rdb"):
    os.remove(f"{os.getcwd()}/dump.rdb")

# Start Redis server with master build
artifact = f"{prefix}{workdir}/bin/linux-x64-debug/search-community/redisearch.so"
os.system(f"redis-server --loadmodule {artifact} --port {master_port}&")
time.sleep(5)  # Give Redis some time to start up

# Load data into Redis
master_redis_client = redis.Redis(host='localhost', port=master_port)
master_redis_client.execute_command("FLUSHALL")

198127:C 07 Apr 2025 04:18:45.784 # WARNING Memory overcommit must be enabled! Without it, a background save or replication may fail under low memory condition. Being disabled, it can also cause failures without low memory condition, see https://github.com/jemalloc/jemalloc/issues/1328. To fix this issue add 'vm.overcommit_memory = 1' to /etc/sysctl.conf and then reboot or run the command 'sysctl vm.overcommit_memory=1' for this to take effect.
198127:C 07 Apr 2025 04:18:45.784 * oO0OoO0OoO0Oo Redis is starting oO0OoO0OoO0Oo
198127:C 07 Apr 2025 04:18:45.784 * Redis version=255.255.255, bits=64, commit=3cdb8c60, modified=0, pid=198127, just started
198127:C 07 Apr 2025 04:18:45.784 * Configuration loaded
198127:M 07 Apr 2025 04:18:45.784 * monotonic clock: POSIX clock_gettime
198127:M 07 Apr 2025 04:18:45.785 * Running mode=standalone, port=6379.
198127:M 07 Apr 2025 04:18:45.788 * <search> Redis version found by RedisSearch : 255.255.255 - oss
198127:M 07 Apr 2025 04:18:45.788 * <sear

198127:M 07 Apr 2025 04:18:50.795 * DB saved on disk


True

In [4]:
prefix = "/home/ubuntu/code/"
workdir = "RediSearch"  # branch build

# Remove dump file if it exists
if os.path.exists(f"{os.getcwd()}/dump.rdb"):
    os.remove(f"{os.getcwd()}/dump.rdb")

# Start Redis server with master build
artifact = f"{prefix}{workdir}/bin/linux-x64-debug/search-community/redisearch.so"
os.system(f"redis-server --loadmodule {artifact} --port {branch_port} &")
time.sleep(5)  # Give Redis some time to start up

# Load data into Redis
branch_redis_client = redis.Redis(host='localhost', port=branch_port)
branch_redis_client.execute_command("FLUSHALL")

198136:C 07 Apr 2025 04:18:53.397 # WARNING Memory overcommit must be enabled! Without it, a background save or replication may fail under low memory condition. Being disabled, it can also cause failures without low memory condition, see https://github.com/jemalloc/jemalloc/issues/1328. To fix this issue add 'vm.overcommit_memory = 1' to /etc/sysctl.conf and then reboot or run the command 'sysctl vm.overcommit_memory=1' for this to take effect.
198136:C 07 Apr 2025 04:18:53.397 * oO0OoO0OoO0Oo Redis is starting oO0OoO0OoO0Oo
198136:C 07 Apr 2025 04:18:53.397 * Redis version=255.255.255, bits=64, commit=3cdb8c60, modified=0, pid=198136, just started
198136:C 07 Apr 2025 04:18:53.397 * Configuration loaded
198136:M 07 Apr 2025 04:18:53.398 * monotonic clock: POSIX clock_gettime
198136:M 07 Apr 2025 04:18:53.398 * Running mode=standalone, port=6378.
198136:M 07 Apr 2025 04:18:53.401 * <search> Redis version found by RedisSearch : 255.255.255 - oss
198136:M 07 Apr 2025 04:18:53.401 * <sear

True

In [5]:
clients = [branch_redis_client, master_redis_client]

In [6]:

# Create RediSearch index - updated to reflect that CID is used in doc name, not as a field
index_cmd = f"""
FT.CREATE idx ON HASH PREFIX 1 doc:
SCHEMA
    t TEXT
"""
for client in clients:
    client.execute_command(index_cmd)
    print(f"Created search index: idx")

Created search index: idx
Created search index: idx


In [7]:
def populate_dbs(num_docs, clients, offset):
    fake = Faker()
    pipes = [c.pipeline() for c in clients]
    for i in tqdm(range(num_docs)):
        for pipe in pipes:
            text = fake.sentence().encode("utf-8")
            pipe.hset(f"doc:{i+offset}", mapping={'t'.encode("utf-8"): text})
            if (i + 1) % 10000 == 0:  # Execute pipeline every 1000 commands
                pipe.execute()
        for pipe in pipes:
            pipe.execute()

In [ ]:
import threading
import timeit
from pprint import pprint
import json

# Parameters
query = 'FT.AGGREGATE idx * TIMEOUT 0'
total_docs = 5000001
step = 300000
num_docs = 0
results = {"master": {}, "branch": {}}

def timed_query(client, label, num_docs):
    t = timeit.timeit('client.execute_command(query)', globals={'client': client, 'query': query}, number=5)
    results[label][num_docs] = t

while num_docs < total_docs:
    # Populate DBs synchronously (assuming you want to do this before each measurement)
    populate_dbs(step, [master_redis_client, branch_redis_client], num_docs)
    num_docs += step

    # Create threads for timing
    t1 = threading.Thread(target=timed_query, args=(master_redis_client, "master", num_docs))
    t2 = threading.Thread(target=timed_query, args=(branch_redis_client, "branch", num_docs))

    # Start both threads
    t1.start()
    t2.start()

    # Wait for both to finish
    t1.join()
    t2.join()

    with open(f"results_{num_docs}_json", "w") as f:
        json.dump(results, f, indent=4)

    # Print results
    pprint(results)

with open("final_results.json", "w") as f:
    json.dump(results, f, indent=4)


  0%|          | 342/300000 [00:00<01:27, 3414.92it/s]

 18%|█▊        | 54243/300000 [00:13<01:09, 3537.89it/s]

198127:M 07 Apr 2025 04:19:51.042 * 10000 changes in 60 seconds. Saving...
198127:M 07 Apr 2025 04:19:51.042 * Background saving started by pid 198218
198218:C 07 Apr 2025 04:19:51.108 * DB saved on disk
198218:C 07 Apr 2025 04:19:51.108 * Fork CoW for RDB: current 2 MB, peak 2 MB, average 2 MB
198127:M 07 Apr 2025 04:19:51.143 * Background saving terminated with success


 29%|██▉       | 87022/300000 [00:21<00:46, 4624.56it/s]

198136:M 07 Apr 2025 04:19:59.044 * 10000 changes in 60 seconds. Saving...
198136:M 07 Apr 2025 04:19:59.045 * Background saving started by pid 198221
198221:C 07 Apr 2025 04:19:59.154 * DB saved on disk
198221:C 07 Apr 2025 04:19:59.154 * Fork CoW for RDB: current 5 MB, peak 5 MB, average 3 MB


...

202488:C 07 Apr 2025 17:30:14.681 * DB saved on disk
202488:C 07 Apr 2025 17:30:14.779 * Fork CoW for RDB: current 34 MB, peak 34 MB, average 18 MB


 71%|███████   | 211546/300000 [01:10<00:24, 3610.57it/s]

198136:M 07 Apr 2025 17:30:15.004 * Background saving terminated with success


100%|██████████| 300000/300000 [01:39<00:00, 3019.57it/s]


198136:M 07 Apr 2025 18:17:47.529 * 100 changes in 300 seconds. Saving...
198136:M 07 Apr 2025 18:17:47.647 * Background saving started by pid 202697
202697:C 07 Apr 2025 18:17:52.336 * DB saved on disk
202697:C 07 Apr 2025 18:17:52.439 * Fork CoW for RDB: current 315 MB, peak 315 MB, average 162 MB
198127:M 07 Apr 2025 18:20:53.865 * 100 changes in 300 seconds. Saving...
198127:M 07 Apr 2025 18:20:53.889 * Background saving started by pid 202700
202700:C 07 Apr 2025 18:20:58.434 * DB saved on disk
202700:C 07 Apr 2025 18:20:58.454 * Fork CoW for RDB: current 315 MB, peak 315 MB, average 178 MB
198136:M 07 Apr 2025 19:04:53.482 * Background saving terminated with success
198127:M 07 Apr 2025 19:10:39.690 * Background saving terminated with success
